In [9]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models, Input, Model
from tensorflow.keras.layers import MaxPooling2D

In [ ]:
# COMP3710 Project
# Question 4: Segment the ISICs data set with the Improved UNet [1] with all labels having a minimum Dice similarity
# coefficient of 0.8 on the test set. [Normal Difficulty]

# Student Name: Xiao Sun
# Studeng Number: 45642586


In [40]:


img_GroundTruth = os.listdir(r'C:\Users\s4564258\.keras\datasets\ISIC2018_Task1_Training_GroundTruth_x2')
img_input = os.listdir(r'C:\Users\s4564258\.keras\datasets\ISIC2018_Task1-2_Training_Input_x2')

len(img_GroundTruth), len(img_input)


(2596, 2596)

In [41]:
img_GroundTruth[0], img_GroundTruth[-1]

('ATTRIBUTION.txt', 'LICENSE.txt')

In [47]:


def load_data():
    
    
    # we use img_input[1:-1] because the first and last file is not image document.
    #load input images and process into tf dataset.
    img_input = os.listdir(r'C:\Users\s4564258\.keras\datasets\ISIC2018_Task1-2_Training_Input_x2')
    img_input = [os.path.join(r'C:\Users\s4564258\.keras\datasets\ISIC2018_Task1-2_Training_Input_x2', path) for path in img_input[1:-1]]
    path_img_input = tf.data.Dataset.from_tensor_slices(img_input)
    image_input_ds = path_img_input.map(data_processing_norm_input, num_parallel_calls=AUTOTUNE)
    
    
    #load mask images and process into tf dataset.
    img_GroundTruth = os.listdir(r'C:\Users\s4564258\.keras\datasets\ISIC2018_Task1_Training_GroundTruth_x2')
    img_GroundTruth = [os.path.join(r'C:\Users\s4564258\.keras\datasets\ISIC2018_Task1_Training_GroundTruth_x2', path) for path in img_GroundTruth[1:-1]]
    path_img_GroundTruth = tf.data.Dataset.from_tensor_slices(img_GroundTruth)
    image_mask_ds = path_img_GroundTruth.map(data_processing_norm_GT, num_parallel_calls=AUTOTUNE)
    
    
    # implot_show(image_input_ds.take(4))
    # implot_show(image_mask_ds.take(4))
    
    return image_input_ds, image_mask_ds
    
def data_processing_norm_input(image):
    # process input img data into tf tensor, and normalization.
    
    img_raw = tf.io.read_file(image)
    image = tf.image.decode_jpeg(img_raw, channels=3)
    image = tf.image.resize(image, [192, 192])
    image /= 255.0  # normalize to [0,1] range
    
    return image
    
    
def data_processing_norm_GT(image):
    # process mask (GroundTruth) img data into tf tensor, and normalization.
    
    img_raw = tf.io.read_file(image)
    image = tf.image.decode_jpeg(img_raw, channels=1)
    image = tf.image.resize(image, [192, 192])
    image /= 255.0  # normalize to [0,1] range
    
    return image
    
def implot_show(ds):
    # using imshow to vertify correctly load and process data
    
    plt.figure(figsize=(8,8))
    for n, image in enumerate(ds):
        print(image.shape)
        plt.subplot(2,2,n+1)
        plt.imshow(image)
        plt.grid(False)
        plt.xticks([])
        plt.yticks([])
        plt.show()
        

In [49]:
def split_train_test_val(image_ds):
    # split the whole tf data set into train, validation and test.
    
    # this step will slow down the process.
    # size = len(list(image_ds))
    size = 2594
    
    train_size = int(0.7 * size)
    val_size = int(0.15 * size)
    test_size = int(0.15 * size)
    
    train_image = image_ds.take(train_size)
    val_image = image_ds.skip(train_size)
    test_image = val_image.take(test_size)
    val_image = val_image.skip(test_size)
    
    return train_image, val_image, test_image

In [52]:
image_input_ds, image_mask_ds = load_data()

input_train, input_val, input_test = split_train_test_val(image_input_ds)
mask_train, mask_val, mask_test = split_train_test_val(image_mask_ds)


In [ ]:
def UNet_context_module(filters, inp, layer_name):
    # Each context_module consists of two 3x3 conv layers and a dropout(0.3) in between.
    
    x1 = layers.Conv2D(filters, kernel_size =3, padding = 'same')(inp)
    # x1 = layers.BatchNormalization()(x1)
    x1 = layers.Activation('relu')(x1)
    x1 = layers.layers.Dropout(.3)(x1)
    x2 = layers.Conv2D(filters, kernel_size =3, padding = 'same')(x1)
    # x2 = layers.BatchNormalization()(x2)
    x2 = layers.Activation('relu')(x2)
    x2 = layers.layers.Dropout(.3)(x2)

    return x2
    
def UNet_upsampling_module():
    # ...It is like a layer that combines the UpSampling2D and Conv2D layers into one layer. 
    
    # what twice means in paper?
    x1 = layers.UpSampling2D(size=(2,2))(inp)
    x2 = layers.Conv2D(filters, kernel_size =3, padding = 'same')(x1)
    
    return x2
    
    
def UNet_localization_module():
    # A localization module consists of a 3x3x3 convolution followed by a 1x1x1 convolution that halves the
    # number of feature maps.
    
    x1 = layers.Conv2D(filters, kernel_size =3, padding = 'same')(inp)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Activation('relu')(x1)
    x2 = layers.Conv2D(filters, kernel_size =1, padding = 'same')(x1)
    x2 = layers.BatchNormalization()(x2)
    x2 = layers.Activation('relu')(x2)
    
    return x2
    
    

def UNet_segmentation_module():
    
    

In [ ]:
def Improved_UNet_model():
    
    
    


In [44]:
image_input_ds, image_mask_ds
len(list(image_input_ds))

2594